In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25773006
paper_name = 'du_jiang_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, names=['genes'])

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 30 x 1


In [7]:
original_data['genes'] = original_data['genes'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [9]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['genes'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])

In [11]:
print(original_data.loc[~t,])

Empty DataFrame
Columns: [genes, orfs]
Index: []


In [12]:
original_data['data'] = -1

In [13]:
original_data.set_index('orfs', inplace=True)
original_data.index.name = 'orf'

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(30, 1)

# Load & process tested strains

In [17]:
tested = pd.read_excel('raw_data/DELETION LIBRARY.xlsx', sheet_name='DELETION LIBRARY', skiprows=1)
tested['orf'] = tested['ORF name'].astype(str)
tested['orf'] = clean_orf(tested['orf'])
tested.loc[tested['orf'] == 'YELOO1C','orf'] = 'YEL001C'
tested['orf'] = translate_sc(tested['orf'], to='orf')
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t])


             record no. ORF name   strain batch  plate row  column RG NOTES  \
index_input                                                                   
346               30827   YMR41W  HOM DIP  13_5  304.0   F     6.0      NaN   

            QC NOTES     orf  
index_input                   
346              NaN  YMR41W  


In [18]:
tested = tested.loc[t,:]

In [19]:
tested_orfs = np.unique(tested['orf'].values)

In [20]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [21]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [16461]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,16461
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16461
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16461       
data_type       value valuez
gene_id orf                 
2       YAL002W     0    0.0
1863    YAL004W     0    0.0
4       YAL005C     0    0.0
5       YAL007C     0    0.0
6       YAL008W     0    0.0

# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 25773006...


  0%|          | 0/1 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.46s/it]

Updating the data_modified_on field...
